# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Casey Chadwell

## SID: 3033291861



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [143]:
import requests 
import bs4 as bs 
import pandas as pd
import numpy as np
import re
from collections import Counter

## 1.1: Links to First Presidential Debates

In [168]:
source = requests.get("https://www.debates.org/voter-education/debate-transcripts/") 
soup = bs.BeautifulSoup(source.content, features='html.parser') 
all_a = soup.find_all('a')

links_to_years = []
titles = []

for a in all_a:
    if re.search(r'(1988|1984|1976|1960): The First', a.contents[0]):
        link = a.get('href')
        links_to_years.append(link)
        titles.append(a.contents[0])
        
for link in links_to_years:
    print(re.findall(r'(1988|1984|1976|1960)', link)[0], ': https://www.debates.org' + link)

1988 : https://www.debates.org/voter-education/debate-transcripts/september-25-1988-debate-transcript/
1984 : https://www.debates.org/voter-education/debate-transcripts/october-7-1984-debate-transcript/
1976 : https://www.debates.org/voter-education/debate-transcripts/september-23-1976-debate-transcript/
1960 : https://www.debates.org/voter-education/debate-transcripts/september-26-1960-debate-transcript/


## 1.2: Data Frame and Statistics

##### A. titles as columns

In [169]:
# got titles above in for loop
titles

['September 25, 1988: The First Bush-Dukakis Presidential Debate',
 'October 7, 1984: The First Reagan-Mondale Presidential Debate',
 'September 23, 1976: The First Carter-Ford Presidential Debate',
 'September 26, 1960: The First Kennedy-Nixon Presidential Debate']

B. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include \ characters in your count, but remove any breakline characters, i.e. \n. You will get credit if your count is +/- 10% from our result.



In [170]:
def get_char_count(url, div_id):
    source = requests.get(url)
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    all_divs = soup.find(id = div_id).text
    return(len(str(all_divs).replace('\n', '')))

counts = []

for link in links_to_years:
    counts.append(get_char_count('https://www.debates.org' + link, 'content-sm'))

counts

[87488, 86505, 80735, 60937]

C. Count how many times the word war was used in the different debates. Note that you have to convert the text in a smart way (to not count the word warranty for example, but counting war., war!, war, or War etc.

In [172]:
def get_war_count(url, div_id):
    source = requests.get(url)
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    all_divs = soup.find(id = div_id).text
    return(len(re.findall(' war ', re.sub(r"[^\w\s]+?", "", str(all_divs).lower()))))

war_counts = []

for link in links_to_years:
    war_counts.append(get_war_count('https://www.debates.org' + link, 'content-sm'))

war_counts

[7, 2, 5, 3]

D. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.

In [173]:
def get_common_count(url, div_id):
    source = requests.get(url)
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    all_divs = soup.find(id = div_id).text
    words = re.findall('\w+', re.sub(r"[^\w\s]+?", "", str(all_divs).lower()))
    Counter(words).most_common(10)
    return(Counter(words).most_common(1))
    

common_w = []
common_c = []

for link in links_to_years:
    common = get_common_count('https://www.debates.org' + link, 'content-sm')
    common_w.append(common[0][0])
    common_c.append(common[0][1])

print(common_w)
print(common_c)

['the', 'the', 'the', 'the']
[799, 867, 856, 779]


#### 1.2 End Solution: Data Frame

In [174]:
pd.DataFrame(data = [counts, war_counts, common_w, common_c], columns = titles, index = ['Debate char length', 'war_count', 'most_common_w', 'most_common_w_count'])

,"September 25, 1988: The First Bush-Dukakis Presidential Debate","October 7, 1984: The First Reagan-Mondale Presidential Debate","September 23, 1976: The First Carter-Ford Presidential Debate","September 26, 1960: The First Kennedy-Nixon Presidential Debate"
Debate char length,87488,86505,80735,60937
war_count,7,2,5,3
most_common_w,the,the,the,the
most_common_w_count,799,867,856,779


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [208]:
base_link = "http://people.sc.fsu.edu/~jburkardt/datasets/regression/"
source = requests.get(base_link) 
soup = bs.BeautifulSoup(source.content, features='html.parser') 
all_a = soup.find_all('a')

links_to_data = []
lines = []

for a in all_a:
    if re.search(r'(x[0-1][0-9].txt)|(x2[0-7].txt)', a.contents[0]):
        lines = []
        link = a.get('href')
        links_to_data.append(base_link + link)

for link in links_to_data:
    source = requests.get(link) 
    soup = bs.BeautifulSoup(source.content, features='html.parser') 
line_5 = re.findall('#.*#', str(soup.text))
lines.append(line_5)
print(line_5)

[]


In [209]:
line_5 = re.findall('#.*#', str(soup.text))
lines.append(line_5)
print(line_5)

[]


In [193]:
for author in line_5:
    counter()
    


'#  x27.txt\n#\n#  Reference:\n#\n#    S C Narula, J F Wellington,\n#    Linear Regression and the Minimum Sum of Relative Errors,\n#    Technometrics, Volume 19, 1977, pages 185-190.\n#\n#    Helmut Spaeth,\n#    Mathematical Algorithms for Linear Regression,\n#    Academic Press, 1991,\n#    ISBN 0-12-656460-4.\n#\n#  Discussion:\n#\n#    The selling price of houses is to be represented as a function of\n#    other variables.\n#\n#    There are 28 rows of data.  The data includes:\n#\n#      I,   the index;\n#      A0,  1;\n#      A1,  the local selling prices, in hundreds of dollars;\n#      A2,  the number of bathrooms;\n#      A3,  the area of the site in thousands of square feet;\n#      A4,  the size of the living space in thousands of square feet;\n#      A5,  the number of garages;\n#      A6,  the number of rooms;\n#      A7,  the number of bedrooms;\n#      A8,  the age in years;\n#      A9,  1 = brick, 2 = brick/wood, 3 = aluminum/wood, 4 = wood.\n#      A10, 1 = two story,

In [ ]:
https://github.com/caseyc49s/casey_chadwell_dataX/blob/master/HW7%20-%20Webscraping.ipynb